In [50]:
"""This is for the 64 bit windows Ballard 1553 BTIDriver.
Other systems have different types for hCore, and hCard"""
from contextlib import contextmanager
import time
import logging
from ctypes import *
import ctypes
import json

class EventLogOptions:
    LOGCFG_DEFAULT = 0
    LOGCFG_ENABLE = 0
    LOGCFG_DISABLE = 1
    
class MessageOptions:
    MSGCRT1553_DEFAULT = 0x00000000  # Default settings
    MSGCRT1553_ENABLE = 0x00000000  # Enable subaddress (RT) (default)
    MSGCRT1553_DISABLE = 0x00001000  # Disable subaddress (RT)
    MSGCRT1553_RESP = 0x00000000  # Enable response (RT) (default)
    MSGCRT1553_NORESP = 0x00002000  # Disable response (RT)
    MSGCRT1553_NOWRAP = 0x00000000  # Disable data wrap (RT) (default)
    MSGCRT1553_WRAP = 0x00004000  # Enable data wrap (RT)
    MSGCRT1553_EOF = 0x00000000  # Message will be transmitted at the end of the frame (default)
    MSGCRT1553_HIPRI = 0x00010000  # Message transmission will interrupt scheduled messages
    MSGCRT1553_NOLOG = 0x00000000  # Message will not generate event log (default)
    MSGCRT1553_LOG = 0x00000001  # Message will generate event log
    MSGCRT1553_NOERR = 0x00000000  # No message error will be generated for message (default)
    MSGCRT1553_ERR = 0x00000002  # A message error will be generated for message
    MSGCRT1553_BUSA = 0x00000000  # Message will be transmitted on bus A (BC) (default)
    MSGCRT1553_BUSB = 0x00000004  # Message will be transmitted on bus B (BC)
    MSGCRT1553_BCRT = 0x00000000  # Message is a BC-RT transfer (BC) (default)
    MSGCRT1553_RTRT = 0x00000008  # Message is an RT-RT transfer (BC)
    MSGCRT1553_NOMON = 0x00000000  # Message will not be monitored (default)
    MSGCRT1553_MON = 0x00000010  # Message will be monitored
    MSGCRT1553_NOTIMETAG = 0x00000000  # Message will not record time-tag (default)
    MSGCRT1553_TIMETAG = 0x00000040  # Message will record time-tag
    MSGCRT1553_NOELAPSE = 0x00000000  # Message will not record elapse time (default)
    MSGCRT1553_ELAPSE = 0x00000080  # Message will record elapse time
    MSGCRT1553_NOMIN = 0x00000000  # Message will not record min time (default)
    MSGCRT1553_MIN = 0x00000100  # Message will record min time
    MSGCRT1553_NOMAX = 0x00000000  # Message will not record max time (default)
    MSGCRT1553_MAX = 0x00000200  # Message will record max time
    MSGCRT1553_NOHIT = 0x00000000  # Message will not record hit count (default)
    MSGCRT1553_HIT = 0x00000400  # Message will record hit count
    MSGCRT1553_NOSYNC = 0x00000000  # No sync will be generated for message (default)
    MSGCRT1553_SYNC = 0x40000000  # Sync will be generated for message
    MSGCRT1553_WIPE = 0x00000000  # Enables message clear (default)
    MSGCRT1553_NOWIPE = 0x80000000  # Disables message clear
    MSGCRT1553_WIPE0 = 0x00000000  # Initialize data with zeros (default)
    MSGCRT1553_WIPE123 = 0x01000000  # Initialize data with incrementing values
    MSGCRT1553_WIPECWD = 0x02000000  # Initialize data with command word

class RTOptions:
    "Remote terminal options"
    RTCFG1553_DEFAULT = 0x00000000  # Select all default settings
    RTCFG1553_SIMULATE = 0x00000000  # Enable RT simulation (default)
    RTCFG1553_DISABLE = 0x00000001  # Disable RT
    RTCFG1553_MONITOR = 0x00000002  # Enable RT monitor
    RTCFG1553_NOBCAST = 0x00000000  # Disable broadcast (default)
    RTCFG1553_BCAST = 0x00000100  # Enable broadcast
    RTCFG1553_NOAUTOBUSY = 0x00000000  # Disable auto busy (default)
    RTCFG1553_AUTOBUSY = 0x00000200  # Enable auto busy
    RTCFG1553_BUILD = 0x00000000  # Enable auto building (default)
    RTCFG1553_NOBUILD = 0x00000400  # Disable auto building
    RTCFG1553_STDB = 0x00000000  # Use MIL-STD-1553B standard (default)
    RTCFG1553_STDA = 0x00400000  # Use MIL-STD-1553A standard
    RTCFG1553_NODYNBC = 0x00000000  # Do not respond to dynamic BC mode code (default)
    RTCFG1553_DYNBC = 0x00001000  # Respond to dynamic BC mode code
    RTCFG1553_NOIMMCLR = 0x00000000  # Do not clear status word bits (default)
    RTCFG1553_IMMCLR = 0x00002000  # Status word bits are cleared immediately
    RTCFG1553_NOBCASTADDR = 0x00000000  # Disable broadcast handling for address (default)
    RTCFG1553_BCASTADDR = 0x00004000  # Enable broadcast handling for address
    RTCFG1553_CHANAB = 0x00000000  # Respond to both channels (default)
    RTCFG1553_CHANA = 0x00020000  # Respond to channel A
    RTCFG1553_CHANB = 0x00010000  # Respond to channel B
    RTCFG1553_CHANNONE = 0x00030000  # Respond to neither channel A nor B
    RTCFG1553_MC01 = 0x00000000  # Select SA=00000 or SA=11111 for mode codes (default)
    RTCFG1553_MC1 = 0x01000000  # Select SA=11111 for mode codes
    RTCFG1553_MC0 = 0x02000000  # Select SA=00000 for mode codes
    RTCFG1553_MCNONE = 0x03000000  # Disable mode codes
    RTCFG1553_TERMOFF = 0x00000000  # Direct coupled termination resistance off (Bus A & B) (default)
    RTCFG1553_TERMONA = 0x04000000  # Direct coupled termination resistance on  (Bus A)
    RTCFG1553_TERMONB = 0x08000000  # Direct coupled termination resistance on  (Bus B)
    RTCFG1553_TERMONAB = 0x0C000000  # Direct coupled termination resistance on  (Bus A & B)
    RTCFG1553_SYNCSEL = 0x00000000  # Selective sync (default)
    RTCFG1553_SYNCALL = 0x40000000  # Sync on all messages
    RTCFG1553_WIPE = 0x00000000  # Enables message clear (default)
    RTCFG1553_NOWIPE = 0x80000000  # Disables message clear
    RTCFG1553_WIPE0 = 0x00000000  # Initialize data with zeros (default)
    RTCFG1553_WIPE123 = 0x10000000  # Initialize data with incrementing values
    RTCFG1553_WIPECWD = 0x20000000  # Initialize data with command word
    RTCFG1553_RESPONSEB = 0x00000000  # RT responds using MIL-STD-1553B response time
    RTCFG1553_RESPONSEA = 0x00400000  # RT responds using MIL-STD-1553A response time
    RTCFG1553_SELFTEST = 0x00800000  # This channel will transmit/receive on the internal self-test bus
    RTCFG1553_SELFTESTOFF = 0x00000000  # This channel will transmit/receive on the operational bus (default)


class SequenceOptions:
    """Ballard Sequential Record configuration options"""
    SEQCFG_DEFAULT = 0x00000000  # Select all default settings
    SEQCFG_FILLHALT = 0x00000000  # Enable sequential record in fill and halt mode (default)
    SEQCFG_DISABLE = 0x00000001  # Disable sequential record
    SEQCFG_CONTINUOUS = 0x00000002  # Enable sequential record in continuous mode
    SEQCFG_DMA = 0x00000004  # Enable monitor in DMA mode
    SEQCFG_FREE = 0x00000008  # Enable sequential record in free mode
    SEQCFG_DELTA = 0x00000010  # Enable sequential record in delta mode
    SEQCFG_INTERVAL = 0x00000020  # Enable sequential record in interval mode
    SEQCFG_NOLOGFULL = 0x00000000  # Do not generate event log when sequential record is full (default)
    SEQCFG_LOGFULL = 0x00001000  # Generate event log when sequential record is full
    SEQCFG_NOLOGFREQ = 0x00000000  # Do not generate event logs at a user specified frequency (default)
    SEQCFG_LOGFREQ = 0x00002000  # Generate event logs at user specified frequency
    SEQCFG_TCPNODELAY = 0x00004000  # Disable Nagle's algorithm on sequential DMA for RPC devices
    SEQCFG_16K = 0x00000000  # Allocate a 16K sequential record buffer (default)
    SEQCFG_ALLAVAIL = 0x01000000  # Allocate all available memory to a sequential record buffer
    SEQCFG_32K = 0x02000000  # Allocate a 32K sequential record buffer
    SEQCFG_64K = 0x04000000  # Allocate a 64K sequential record buffer
    SEQCFG_128K = 0x08000000  # Allocate a 128K sequential record buffer


class MonitorOptions:
    """monitor configuration options"""
    MONCFG1553_ENABLE = 0x00000000  # Enable monitor
    MONCFG1553_DISABLE = 0x00000001  # Disable monitor
    MONCFG1553_DEFAULT = 0x00000000  # Select all default settings
    MONCFG1553_NOBCAST = 0x00000000  # Disable broadcast (default)
    MONCFG1553_BCAST = 0x00000100  # Enable broadcast
    MONCFG1553_COMPLETE = 0x00000000  # Reserved
    MONCFG1553_INCOMPLETE = 0x00010000  # Reserved
    MONCFG1553_MC01 = 0x00000000  # Select SA=00000 or SA=11111 for mode codes (default)
    MONCFG1553_MC1 = 0x00100000  # Select SA=11111 for mode codes
    MONCFG1553_MC0 = 0x00200000  # Select SA=00000 for mode codes
    MONCFG1553_MCNONE = 0x00300000  # Disable mode codes
    MONCFG1553_TERMOFF = 0x00000000  # Direct coupled termination resistance off (Bus A & B) (default)
    MONCFG1553_TERMONA = 0x04000000  # Direct coupled termination resistance on  (Bus A)
    MONCFG1553_TERMONB = 0x08000000  # Direct coupled termination resistance on  (Bus B)
    MONCFG1553_TERMONAB = 0x0C000000  # Direct coupled termination resistance on  (Bus A & B)
    MONCFG1553_SELFTEST = 0x00800000  # This channel will transmit/receive on the internal self-test bus
    MONCFG1553_SELFTESTOFF = 0x00000000  # This channel will transmit/receive on the operational bus (default)


CH0 = 0
CH1 = 1
RCV = 0
XMT = 1

class Std1553Exception(BaseException):
    pass

class PcapWriter:
    def __init__(self, file_prefix, stream_count=4):
        file_date = datetime.datetime.now().isoformat()
        for file_num in range(1000):  # max tests per day.
            filename = f'{file_prefix}{file_date}_{file_num}.pcap'
            if not os.path.exists(filename):
                break
        else:
            # none of the filenames are avaliable
            raise Std1553Exception(f'could not open 1553 output pcap file: {file_name}')
        self.fh = open(filename, "wb")
        self.fh.write(self.file_header())
        self.close = self.fh.close

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.fh.close()

    def write_frame(self, frame):
        self.fh.write(self.packet_header(len(frame)))
        self.fh.write(frame)

    def file_header(self):
        """
        Returns: pcap file header in bytes
        """
        magic_number = 0xa1b2c3d4
        timezone = int(time.timezone / 3600)
        version_major = 2
        version_minor = 4
        sigfigs = 6  # time resolution
        snaplen = 1e6  # maximum packet len
        net_type = 1  # use ethernet type
        return struct.pack(
            '<LHHlLLL',
            magic_number,
            version_major,
            version_minor,
            timezone,
            sigfigs,
            snaplen,
            net_type
        )

    def packet_header(self, length):
        seconds, us = divmod(time.time(), 1)
        seconds = int(seconds)
        us = int(us * 1e6)
        return struct.pack(
            '<LLLL',
            seconds, us, length, length
        )
    
class Std1553:
    def __init__(self, logger=None):
        self.logger = logger or logging.getLogger(__name__)
        dll_card = "bticard64"
        dll_1553 = "bti155364"
        self.lib_card = ctypes.windll.LoadLibrary(dll_card)
        self.lib_1553 = ctypes.windll.LoadLibrary(dll_1553)
        self.NUM_CORES=2
        self.core_nums = [c_int(1),c_int(0)]
        self.card_num = c_int(1)
        # only correct for win64
        self.hCores = [pointer(ctypes.c_long()) for _ in range(self.NUM_CORES)]
        self.hCard = pointer(ctypes.c_long())
        BufferType = c_uint16 * 2000
        self.buffer = BufferType()
        self.addresses = range(1,30)
        self.messages = list()
        
    def __enter__(self):
        self.logger.debug(f'before cores:{[_.contents for _ in self.hCores]}')
        self.open()
        self.logger.debug(f'after cores:{_.contents for _ in self.hCores}')
        return self
    def __exit__(self, *exc):
        self.close()
    def check_return(self, val):
        if val != 0:
            self.lib_card.BTICard_ErrDescStr.restype = ctypes.c_char_p
            self.lib_card.BTICard_ErrDescStr.argtypes = [ctypes.c_long, ctypes.c_long]
            _=self.lib_card.BTICard_ErrDescStr(val,0).decode()
            raise Std1553Exception(_)   
            
    def open(self):
        hCard = self.hCard
        default = 0
        try:
            self.check_return(self.lib_card.BTICard_CardOpen(byref(hCard), 0))
        except Std1553Exception:
            self.check_return(self.lib_card.BTICard_CardOpen(byref(hCard), 1))
        for core_number in self.core_nums:
            core = core_number
            core_number = core_number.value
            hCore = self.hCores[core_number]
            self.logger.debug(f'unopened core:{core}, hCard:{hCard}')

            self.check_return(self.lib_card.BTICard_CoreOpen(byref(hCore), core, hCard))
            self.logger.debug(f'opened core:{core}, hCard:{hCard}')
            
    def init_rt(self):
        # each core has 2 channels
        for core_number, hCore in enumerate(self.hCores):
            # reset the core.
            self.check_return(self.lib_card.BTICard_CardReset(hCore)) 
            for channel in range(2):
                for address in self.addresses:
                    subaddress = address
                    # each address responds to one subaddress the same
                    # number as the address.
                    self.rt_config(hCore, channel, address)
                    # enable wrap around
                    _ = self.rt_message_create(core_number, channel, address, subaddress, is_transmit=False, 
                                           message_options=MessageOptions.MSGCRT1553_WRAP+MessageOptions.MSGCRT1553_LOG)
                    self.messages.append(_)

    def start(self):
        self.check_return(self.lib_card.BTICard_CardStart(self.hCard))
        
    def rt_config(self, hCore, channel, address, rt_config=RTOptions.RTCFG1553_DEFAULT):
        self.check_return(self.lib_1553.BTI1553_RTConfig(
            c_int(rt_config) ,  # control flags
            c_int(address),       # terminal address
            c_int(channel), # channel number
            hCore           # core handle
        ))
        
    def sequential_block_read(self):
        "used for bus monitors"
        block_count = ctypes.c_uint32()
        buffer_size = ctypes.c_uint32(2048)
        array_type = ctypes.c_uint16 * buffer_size.value
        sequence_buffer = array_type()
        result=[]
        "read messages from all cores"
        for hCore in self.hCores:
            sequence_count = self.lib_card.BTICard_SeqBlkRd(
                    sequence_buffer, buffer_size, byref(block_count), hCore
            )
            if block_count.value > 0:
                std1553_data = [hex(sequence_buffer[i]) for i in range(block_count.value)]
                result.append(std1553_data)
                self.logger.debug(f'hCore {hCore.value} read 1553: {std1553_data}')
            if sequence_count != 0 and sequence_count != None:
                last_sequence_count = sequence_count
                self.logger.info(f"1553 sequence count: {sequence_count}")
                    
        return result
    
    def get_message(self, hCore, channel, address, subaddress, is_mode_code=False, is_transmit=False):
        # self.lib_1553.BTI1553_RTGetMsg(SUBADDRESS,1,RCV,2,channel,hCore)
        rt_msg = self.lib_1553.BTI1553_RTGetMsg(
            c_bool(is_mode_code),  # TRUE if mode code
            c_int(address), # Terminal address containing message
            c_bool(is_transmit),      # transmit/receive bit
            c_int(subaddress), # Subaddress or mode-code number
            c_int(channel), # Channel number
            hCore           # Core handle
        )
        return rt_msg

    def rt_message_create(self, core_number, channel, address, subaddress, message_options=MessageOptions.MSGCRT1553_WRAP, is_mode_code=False, is_transmit=True):
        """message creation is only necessary if the rt is initalized with RTCFG1553_NOBUILD
        if the message already exists, it is reconfigured with controlflags."""
        hCore = self.hCores[core_number]
        #self.logger.debug(f'new message core:{core_number}ch:{channel}a:{address}')
        message_address = self.lib_1553.BTI1553_RTCreateMsg(
            ctypes.c_ulong(message_options),  # select config options
            ctypes.c_bool(is_mode_code),   # true if modecode
            ctypes.c_int(address),         # terminal address of message
            ctypes.c_bool(is_transmit),    # transmit/recieve bit
            ctypes.c_int(subaddress),               # subaddress or m/c number.
            ctypes.c_int(channel),              # channel number
            hCore                          # core handle.
        )
        return message_address
        
    def enable_event_log(self, log_length=30):
        for hCore in self.hCores:
            self.check_return(self.lib_card.BTICard_EventLogConfig(EventLogOptions.LOGCFG_DEFAULT,
                                log_length,
                                hCore))
        
    def close(self):
        hCard = self.hCard
        for hCore in self.hCores:
            self.lib_card.BTICard_CardStop(hCore)
        self.lib_card.BTICard_CardClose(hCard)
        self.logger.info("closed 1553 card")
        
    def read_event_log(self):
        EVENTTYPE_1553MSG = 1
        event_type = c_ushort()
        event_value = c_ulong()
        event_channel = c_int()
        for core_number, hCore in enumerate(self.hCores):
            while True:
                # read all events in buffer.
                event_result = self.lib_card.BTICard_EventLogRd(
                    byref(event_type),
                    byref(event_value),
                    byref(event_channel),
                    hCore)
                if event_result == 0:
                    # buffer is empty
                    break
                if event_type.value == EVENTTYPE_1553MSG:
                    self.dump_event_message(event_value, core_number=core_number, channel=event_channel.value)
                else:
                    self.logger.debug(f"unknown event type:{event_type.value} value:{event_value.value} channel:{event_channel.value}")
            
    def process(self):
        self.read_event_log()
    
    def dump_event_message(self, message_addr, core_number, channel=None):
        data_length = 32
        data = (ctypes.c_short*data_length)()
        hCore = self.hCores[core_number]
        self.lib_1553.BTI1553_MsgDataRd(byref(data),data_length,message_addr,hCore)
        data_list = list(data)
        self.logger.debug(json.dumps({'std1553_data':data_list, 'core': core_number, 'channel':channel}))
        # self.logger.info(f"1553:{data_list}")
        

In [51]:
logging.basicConfig(level=logging.DEBUG)
with Std1553() as s:
    run_time=1
    #s.hCores[0]=s.hCard
    s.init_rt()
    s.enable_event_log()
    s.start()
    end_time = time.time() + run_time
    print(f'cores: {[h.contents for h in s.hCores]} card:{s.hCard.contents}')
    print("msg len",len(s.messages), len(set(s.messages)))
    while time.time() < end_time:
        time.sleep(.1)
        s.process()
    print("complete")

DEBUG:__main__:before cores:[c_long(0), c_long(0)]
DEBUG:__main__:unopened core:c_long(1), hCard:<ctypes.wintypes.LP_c_long object at 0x000001EEA2279048>
DEBUG:__main__:opened core:c_long(1), hCard:<ctypes.wintypes.LP_c_long object at 0x000001EEA2279048>
DEBUG:__main__:unopened core:c_long(0), hCard:<ctypes.wintypes.LP_c_long object at 0x000001EEA2279048>
DEBUG:__main__:opened core:c_long(0), hCard:<ctypes.wintypes.LP_c_long object at 0x000001EEA2279048>
DEBUG:__main__:after cores:<generator object Std1553.__enter__.<locals>.<genexpr> at 0x000001EEA22909E8>


cores: [c_long(1645546600), c_long(1645546600)] card:c_long(1645565056)
msg len 116 58


INFO:__main__:closed 1553 card


complete


In [41]:
with Std1553() as s:
    run_time=120
    self = s
    hCore = self.hCores[0]
    hCard = self.hCard
    channel=0
    address=1
    subaddress=1
    core_number = 0
    #self.check_return(self.lib_card.BTICard_CardOpen(byref(hCard), self.card_num))
    #self.check_return(self.lib_card.BTICard_CoreOpen(byref(hCore), self.core_num, hCard))
    #self.check_return(self.lib_card.BTICard_CardReset(hCore)) 
    self.init_rt()
    self.rt_config(hCore, channel, address)
    message = self.rt_message_create(
        core_number, channel, address, subaddress,
        message_options=MessageOptions.MSGCRT1553_WRAP+MessageOptions.MSGCRT1553_LOG)
    
    self.check_return(self.lib_card.BTICard_CardStart(hCard))

    # create list buffer replaces create message.
    # self.check_return(self.lib_1553.BTI1553_RTCreateList(   
    # ))
    # BTI1553_ListDataWr
    # BTI1553_ListDataRd
    # for bus monitor read the sequential record from each core.
    # create event log list
    
    
    #log_length = 30
    #self.lib_card.BTICard_EventLogConfig(EventLogOptions.LOGCFG_DEFAULT,
    #                                    log_length,
    #                                    hCore)
    self.enable_event_log()
    start_time = time.time()
    self.logger.debug(f'handle{s.hCard.contents} {s.hCores[0].contents}')
    while (time.time() - start_time) < run_time:
        time.sleep(.1)
        # self.lib_1553.BTI1553_MsgDataRd(byref(data),data_length,message,hCore)
        
        # read event log returns zero if there are no events.
        EVENTTYPE_1553MSG = 1
        event_type = c_ushort()
        event_value = c_ulong()
        event_channel = c_int()
        while True:
            event_result = self.lib_card.BTICard_EventLogRd(
                byref(event_type),
                byref(event_value),
                byref(event_channel),
                hCore)
            if event_result == 0:
                print('.',end="")
                break
            else:
                # type 1 = 1553 message.
                if event_type.value == EVENTTYPE_1553MSG:
                    self.dump_event_message(event_value, core_number=core_number, channel=channel)
                else:    
                    print(f'1553 event log: {event_result} type:{event_type} value: {event_value}, channel:{event_channel}')
                
        data_list = list(data)
#         if data_list == [0,0,0]:
#             print(".",end="")
#         else:
#             print(data_list)
#             break
        #s.process()
    print("complete")

DEBUG:__main__:before cores:[c_long(0), c_long(0)]
DEBUG:__main__:after cores:<generator object Std1553.__enter__.<locals>.<genexpr> at 0x000001EEA2271BF8>


unopened core:c_long(0), hCard:<ctypes.wintypes.LP_c_long object at 0x000001EEA22F3348>
opened core:c_long(0), hCard:<ctypes.wintypes.LP_c_long object at 0x000001EEA22F3348>
unopened core:c_long(1), hCard:<ctypes.wintypes.LP_c_long object at 0x000001EEA22F3348>
opened core:c_long(1), hCard:<ctypes.wintypes.LP_c_long object at 0x000001EEA22F3348>


DEBUG:__main__:handlec_long(1645565056) c_long(1645546600)
INFO:__main__:closed 1553 card


.

NameError: name 'data' is not defined

In [21]:
a[0]


0

In [5]:
!ls

archive
env
env-old
gse1.ipynb
src
stresstest.txt
venv


TypeError: an integer is required (got type c_long)

256